In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('clean_data.csv')
df.head()

,Unnamed: 0,ID,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,multiple_deliveries,Festival,City,Time_taken (min),approx_distance_km,travel_time,distance_km
0,0,0xcdcd,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,3.0,4.0,2,3.0,0.0,3.0,46,10.988439,28.450000,16.8077
1,1,0xd987,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,5.0,3.0,1,1.0,0.0,3.0,23,6.279108,13.115000,8.8577
2,2,0x2784,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,4.0,2.0,1,1.0,0.0,3.0,21,14.127993,18.655000,21.3843
3,3,0xc8b6,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,4.0,1.0,0,0.0,0.0,3.0,20,3.139554,8.308333,4.8283
4,4,0xdb64,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,3.0,4.0,1,1.0,0.0,3.0,41,20.407102,24.350000,23.4788


In [3]:
df.shape

(37909, 19)

In [4]:
df.isnull().sum()

Unnamed: 0                     0
ID                             0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
approx_distance_km             0
travel_time                    3
distance_km                    3
dtype: int64

In [5]:
df.dropna(inplace = True)

In [6]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

In [7]:
features = ['Weather_conditions', 'Road_traffic_density', 'Vehicle_condition', 'multiple_deliveries', 'Festival', 'City', 'distance_km']
print(features)

['Weather_conditions', 'Road_traffic_density', 'Vehicle_condition', 'multiple_deliveries', 'Festival', 'City', 'distance_km']


In [8]:
X = df[features]

In [9]:
y = df['Time_taken (min)']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

LightGBM

In [11]:
from lightgbm import LGBMRegressor

params = {
    'n_estimators': 351,
    'learning_rate': 0.020468063165234227,
    'max_depth': 9,
    'num_leaves': 60,
    'min_child_samples': 47,
    'subsample': 0.6284384317237434,
    'colsample_bytree': 0.7896582974323323,
    'reg_alpha': 0.0019326774481699123,
    'reg_lambda': 0.05070012546687946,
    'random_state': 42
}

model = LGBMRegressor(**params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000729 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 279
[LightGBM] [Info] Number of data points in the train set: 28429, number of used features: 7
[LightGBM] [Info] Start training from score 26.603503
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

Best MAE params: {'n_estimators': 351, 'learning_rate': 0.020468063165234227, 'max_depth': 9, 'num_leaves': 60, 'min_child_samples': 47, 'subsample': 0.6284384317237434, 'colsample_bytree': 0.7896582974323323, 'reg_alpha': 0.0019326774481699123, 'reg_lambda': 0.05070012546687946}
Best MAE score: 4.903850914161457

In [12]:
mae = mean_absolute_error(y_pred, y_test)
print(mae)

4.899000233165826


In [13]:
errors = np.abs(y_test - y_pred)

In [14]:
error_df = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_pred,
    'abs_error': errors
})

worst_preds = error_df.sort_values(by='abs_error', ascending=False).head()
print(worst_preds)

       y_true     y_pred  abs_error
14813      54  29.075342  24.924658
22167      42  18.680670  23.319330
35102      50  27.412601  22.587399
198        43  20.543419  22.456581
15378      52  29.676629  22.323371


In [15]:
tolerance = mae
correct_within_range = np.abs(y_pred - y_test) <= tolerance
custom_accuracy = np.mean(correct_within_range)
print(f"Custom Accuracy (±{tolerance}): {custom_accuracy * 100:.2f}%")

Custom Accuracy (±4.899000233165826): 58.72%


In [16]:
import joblib

joblib.dump(model, 'model.joblib')

['model.joblib']